## Actual Implementation

In [17]:
#import statements
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch_geometric_temporal.nn.recurrent import AGCRN

## initalisation

In [20]:
#try batch size=4/7 for 25 years of historical data; for a seq length of 5(5/3 batches per epoch)
#input features will be input sectors (8) + 3 additional features for now
#T will be every 5 years (tbd)
#assume data will be read from csv 

import pandas as pd

class ModelDataset(Dataset):
    def __init__(self, csv_file, T=5):
        """
        X shape: [num_samples, T, num_nodes, in_channels]
        Y shape: [num_samples, num_nodes, num_sectors]
        """
        df=pd.read_csv(csv_file)
        data = df.values 
        self.T = T
        self.X = []
        self.y = []
        
        # Build sequences of length 'T'
        # For each index i, we take rows[i : i+T] as inputs 
        # and row[i+T] (or some slice) as the target.
        for i in range(len(data) - T):
            # For example, let's say columns [0:13] are features, column 13 is target
            x_seq = data[i : i + T, :13] 
            y_val = data[i + T, 13]

            self.X.append(x_seq)
            self.y.append(y_val)

        # Convert to tensors
        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.float32)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

## Country Dictionary

0. Singapore
1. China
2. Malaysia
3. United States
4. Hong Kong, China
5. Indonesia
6. Korea, Rep.
7. Japan
8. Thailand
9. Australia
10. Vietnam
11. India
12. United Arab Emirates
13. Philippines
14. Germany
15. France
16. Switzerland
17. Netherlands

## Sector Dictionary
0. Category 1 (Agri)
1. Category 2 (Mining)
2. Category 3 (Construction)
3. Category 4 (Textile)
4. Category 5 (Transport Svcs)
5. Category 6 (ICT)
6. Category 7 (Health, pharm, sports etc)
7. Category 8 (Govt, Millitary, Misc)

In [ ]:
num_countries=18
num_country_pairs=18*(17-1) 
num_sectors=8 # 8 sectors

class Args:
    def __init__(self):
        # Model structure
        self.num_nodes = num_country_pairs  
        self.input_dim = num_sectors+3    # e.g. sectorial export volume + sentiment score + 2 indexes
        self.rnn_units = 32
        self.output_dim = num_sectors   # e.g., predict only the sectorial export volume
        self.horizon = 2      # forecast 2 steps ahead
        self.num_layers = 2
        self.cheb_k = 2
        self.embed_dim = 10
        self.default_graph = True  
        self.log_dir = './logs/'
        self.debug = False
        self.model='AGCRN'
        self.normaliser = 'std'
        self.device='cpu'
        self.batch_size=4 # 4/7 depending on results
        
        # Training
        self.seed=10
        self.loss_func= 'mse'
        self.epochs = 10
        self.lr_init = 0.003
        self.lr_decay = False
        self.lr_decay_steps = 5,20,40,70
        self.lr_decay_rate = 0.3
        self.early_stop = True
        self.early_stop_patience = 15
        self.teacher_forcing = False
        self.tf_decay_steps = 2000
        self.real_value = True
        self.grad_norm = True
        self.max_grad_norm = 5

        # Testing
        self.mae_thresh=None
        self.mape_thresh=0.

        #Logging
        self.log_step = 20
        self.plot=False



args = Args()

## Data handling

In [ ]:
# Create the dataset and data loader
train_dataset= ModelDataset('data.csv', T=args.horizon)
val_dataset= ModelDataset('data.csv', T=args.horizon)
test_dataset= ModelDataset('data.csv', T=args.horizon)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)



FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

## training algo

In [ ]:
from datetime import datetime
from AGCRN.model.BasicTrainer import Trainer
from agcrn_model import AGCRNFinal

model=AGCRNFinal(args)
model=model.to(args.device)
# for p in model.parameters():
#     nn.init.xavier_uniform_(p)

#load dataset here

#init loss function, optimizer
loss=torch.nn.MSELoss().to(args.device)
optimizer=optim.Adam(model.parameters(),lr=args.lr_init,eps=1.0e-8,weight_decay=0.0,amsgrad=False)

#learning rate decay
lr_scheduler=None
if args.lr_decay:
    print('Applying learning rate decay.')
    lr_decay_steps = [int(i) for i in list(args.lr_decay_step.split(','))]
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
                                                        milestones=lr_decay_steps,
                                                        gamma=args.lr_decay_rate)

#config log path
current_time = datetime.now().strftime('%Y%m%d%H%M%S')
current_dir = os.getcwd()
log_dir = os.path.join(current_dir,'logs')
args.log_dir = log_dir

#start training
trainer = Trainer(model, loss, optimizer, train_loader, val_loader, test_loader, scaler, #need to get these 
                  args, lr_scheduler=lr_scheduler)
if args.mode == 'train':
    trainer.train()
# elif args.mode == 'test':
#     model.load_state_dict(torch.load('./pre-trained/{}.pth'.format(args.dataset)))
#     print("Load saved model")
#     trainer.test(model, trainer.args, test_loader, scaler, trainer.logger)
# else:
#     raise ValueError


TypeError: expected str, bytes or os.PathLike object, not module

In [ ]:
# # MODEL DEFINITION
# model = AGCRN(
#     number_of_nodes=num_country_pairs,
#     in_channels=in_channels,
#     out_channels=out_channels,
#     K=K,
#     embedding_dimensions=embedding_dims
# )

# # 3) Create the node embedding E separately (following your interface).
# #    We'll just do a random init. This is learnable, so we wrap it in nn.Parameter.
# E = nn.Parameter(torch.zeros(num_country_pairs, embedding_dims), requires_grad=True)

# # 4) "prediction head" to map from [out_channels] -> [num_sectors]
# prediction_head = nn.Linear(out_channels, num_sectors)

# # 5) Combine everything in a single optimizer. We must include the node embedding (E) as well.
# optimizer = optim.Adam(
#     list(model.parameters()) + list(prediction_head.parameters()) + [E],
#     lr=lr
# )

# criterion = nn.MSELoss()

# # 6) Training loop
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0.0

#     for X_batch, Y_batch in dataloader:
#         # X_batch: [batch_size, num_nodes, in_channels]
#         # Y_batch: [batch_size, num_nodes, num_sectors]

#         optimizer.zero_grad()

#         H = None
#         # Unroll over T time steps
#         for t in range(T):
#             X_t = X_batch[:, t, :, :]  # [batch_size, num_nodes, in_channels]
#             H = model(X_t, E, H)  # H is the hidden state, E is the node embedding
            

#         # Now map from [out_channels] -> 1 dimension
#         # We'll do this for each node:
#         Y_pred = prediction_head(H)
#         print('Y_pred.shape', Y_pred.shape)
#         # Compute MSE loss with target
#         loss = criterion(Y_pred, Y_batch)

#         # Backprop & update
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item() * X_batch.size(0)

#     avg_loss = total_loss / len(dataset)
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# print("Training complete!")

NameError: name 'num_country_pair' is not defined